In [6]:
using Genie, Genie.Router, Genie.Requests, Genie.Renderer, Genie.Renderer.Html, Genie.Renderer.Json

In [7]:
@time ary_gcp機器列表=split(read(`gcloud compute instances list`,String),"\n")
int_port=18866
str_本機外部IP=[split(i,r" +") for i in ary_gcp機器列表 if occursin("cameo-motion-2",i)][1][6]
str_url="http://$(str_本機外部IP):$(int_port)/hi?name=BowenChiu"
str_url="http://$(str_本機外部IP):$(int_port)/form"
HTML("""<a href=$(str_url)>$(str_url)</a>""")

  1.895600 seconds (124 allocations: 198.312 KiB)


HTML{String}("<a href=http://35.194.185.192:18866/form>http://35.194.185.192:18866/form</a>")

In [8]:
route("/hello") do
  "Hello - Welcome to Genie!"
end
route("/hello.html") do
  html("Hello World")
end
route("/hello.json") do
  json("Hello World")
end
route("/hello.txt") do
   respond("Hello World", :text)
end
route("/hi") do
  name = haskey(@params, :name) ? @params(:name) : "Anon"
  "Hello $name"
end

[GET] /hi => #31 | :get_hi

In [9]:
form = """
<meta charset="UTF-8">
<form action="/form" method="POST" enctype="multipart/form-data">
  <input type="text" name="name" value="" placeholder="What's your name?" />
  <input type="submit" value="Greet" />
</form>
"""

route("/form") do
  html(form)
end

route("/form", method = POST) do
  html("""<meta charset="UTF-8"> Hello $(postpayload(:name, "Anon"))""")
end

[POST] /form => #35 | :post_form

In [ ]:
Genie.config.run_as_server = true
Genie.config.cors_headers["Access-Control-Allow-Origin"] = "http://localhost:$(int_port)"
Genie.config.cors_headers["Access-Control-Allow-Headers"] = "Content-Type"
Genie.config.cors_headers["Access-Control-Allow-Methods"] ="GET,POST,PUT,DELETE,OPTIONS" 
Genie.config.cors_allowed_origins = ["*"]
up(int_port, "0.0.0.0", async = false, verbose = true)